In [ ]:
from collections import deque
import heapq

#Creation of the chessboard:
#How queens can move
#How to check if they are in a valid position
#How to determine if we have won the game (goal state)

class ModifiedNQueensProblem:
    def __init__(self, N):
        self.N = N
        self.initial_state = tuple((0, col) for col in range(N))

#We use DL (Down-Left) and DR (Down-Right) only for N =< 5, but not for N > 5
#The reason is to reduce complexity and make solving larger N easier.

#For smaller chessboards (N = 4 or N = 5), there is enough space for queens to move diagonally (DL and DR).
#Since there are fewer queens, using diagonal moves helps find a solution faster.
#The problem is small, so we can explore more movement options without making the search too slow.

#For larger chessboards (N = 6, 7, 8...), we only allow straight-down (D) moves.
#If we allow DL and DR for large N, the number of possible states grows exponentially.
#This makes the search very slow because the algorithm has to check too many positions.

#When N is big, queens can spread out easily with just D (straight down).
#DL and DR increase the risk of diagonal conflicts, making it harder to find a safe arrangement.
    
        self.allowed_moves = ['D', 'DL', 'DR'] if N <= 5 else ['D']


#We are moving the queens down from the top row (row 0) to the bottom row (row N-1).
#That's why we only consider downward moves:
# D (Down) -> Move straight down to the next row.
# DL (Down-Left) -> Move down and to the left.
# DR (Down-Right) -> Move down and to the right.

#We didn't use U (Up)
#If we allowed U (Up), queens would move back up, which is not useful in this problem.
#Our goal is to place all queens safely from the first row to the last row.
#If a queen goes back up, it would be undoing progress.

#For N=4
#Let's say we start with queens at: [(0,0), (0,1), (0,2), (0,3)]
#We want to move them down until they reach row 3 safely.
#If we allowed U, a queen could move back up instead of reaching row 3. That wouldn't help solve the problem.

    def actions(self, state):
        possible_actions = []
        occupied_positions = set(state)

        for idx, (row, col) in enumerate(state):
            for direction in self.allowed_moves:
                new_row, new_col = row, col

                if direction == 'D':
                    new_row += 1
                elif direction == 'DL':
                    new_row += 1
                    new_col -= 1
                elif direction == 'DR':
                    new_row += 1
                    new_col += 1

                if (0 <= new_row < self.N and 0 <= new_col < self.N and
                (new_row, new_col) not in occupied_positions):
                    possible_actions.append((idx, direction))

        return possible_actions
    
#This function moves a queen to a new position.
#It takes:
#The current state (positions of all queens).
#The action (which queen moves and where).
#It updates the queen's position and returns the new state.

    def result(self, state, action):
        idx, direction = action
        row, col = state[idx]

        if direction == 'D':
            new_row, new_col = row + 1, col
        elif direction == 'DL':
            new_row, new_col = row + 1, col - 1
        elif direction == 'DR':
            new_row, new_col = row + 1, col + 1

        new_state = list(state)
        new_state[idx] = (new_row, new_col)
        return tuple(new_state)

#This function checks if the game is won.
#It makes sure:
#No two queens are in the same row or same column.
#No two queens are on the same diagonal (like a / or \).

    def goal_test(self, state):
        rows = set()
        cols = set()

        for row, col in state:
            if row in rows or col in cols:
                return False
            rows.add(row)
            cols.add(col)

        # Check diagonal conflicts
        for i in range(self.N):
            for j in range(i + 1, self.N):
                if abs(state[i][0] - state[j][0]) == abs(state[i][1] - state[j][1]):
                    return False

        return True
    
#This function gives a number (score) to a state.
#If queens are clashing, the score is high.
#If queens are safe, the score is low.
#A* search uses this score to find the best path faster.


    def heuristic(self, state):
        if self.N <= 5:
#For small boards (N = 4 or N = 5), we use a simple heuristic that counts the number of conflicts between queens.
#We check every pair of queens (i, j).
#Each conflict adds 2 penalty points to the heuristic.

            # Conflict-based heuristic for N=4,5
            conflicts = 0
            for i in range(self.N):
                for j in range(i + 1, self.N):
                    if (state[i][0] == state[j][0] or
                        state[i][1] == state[j][1] or
                        abs(state[i][0] - state[j][0]) == abs(state[i][1] - state[j][1])):
                        conflicts += 2
            return conflicts
        else:
#For large boards (N >= 6), we used a smarter heuristic to make the search more efficient. Instead of only counting conflicts, we use a weighted sum of three factors:
# 1) HR (Remaining Empty Rows): Counts how many rows don't have queens yet
# 2) HC (Conflict Penalty): Checks how many queens share the same position
# 3) HM (Unique Placements Bonus): Counts how many unique positions the queens are placed in. More unique positions mean a better spread of queens across the board.
            # Hybrid heuristic for N >= 6
            HR = sum(1 for row, col in state if row < self.N - 1)  # Remaining empty rows
            HC = sum(state.count(pos) > 1 for pos in state)  # Conflict penalty
            HM = len(set(state))  # Unique placements bonus

            return HR + HC - 0.5 * HM  # Weighted heuristic

#BFS checks all possible moves level by level.
#It slowly moves the queens down one row at a time.
#It finds the solution if it exists but takes more time.

#working:
#Starts with the initial board state (all queens in row 0).
#Uses a queue (FIFO - First In, First Out) to explore states.
#Expands the current state by applying all valid actions (queen moves).
#Checks if a new state is already visited (explored set).
#If a goal state (valid N-Queens placement) is found, returns the solution path.
#If no solution is found, returns None.

def bfs(problem):
    frontier = deque([(problem.initial_state, [])])
    explored = set()
    nodes_expanded = 0

    while frontier:
        state, path = frontier.popleft()
        if problem.goal_test(state):
            return path, nodes_expanded, len(frontier)

        if state not in explored:
            explored.add(state)
            nodes_expanded += 1

            for action in problem.actions(state):
                new_state = problem.result(state, action)
                if new_state not in explored:
                    frontier.append((new_state, path + [action]))

    return None, nodes_expanded, len(frontier)

#UCS is similar to BFS, but it always expands the cheapest path first.
#Since every move has the same cost here, UCS behaves like BFS.

#working:
#Uses a priority queue (min-heap) to explore the lowest-cost states first.
#The cost is always +1 per move (all moves have equal cost).
#Expands nodes in order of increasing cost.
#If a goal state is reached, returns the solution path.

def ucs(problem):
    frontier = [(0, problem.initial_state, [])]
    explored = set()
    nodes_expanded = 0

    while frontier:
        cost, state, path = heapq.heappop(frontier)
        if problem.goal_test(state):
            return path, nodes_expanded, len(frontier)

        if state not in explored:
            explored.add(state)
            nodes_expanded += 1

            for action in problem.actions(state):
                new_state = problem.result(state, action)
                if new_state not in explored:
                    heapq.heappush(frontier, (cost + 1, new_state, path + [action]))

    return None, nodes_expanded, len(frontier)

#A* is smarter than BFS and UCS.
#It uses the heuristic to decide which path to explore first.
#This helps it find the solution faster.

#working:
#Uses a priority queue (min-heap) like UCS.
#Each move has a cost (g) and a heuristic (h):
#g = cost so far (number of moves)
#h = estimated cost to goal (conflict-based or hybrid heuristic)
#Always expands the state with the lowest total cost f = g + h.
#If a goal state is found, returns the solution path.

def astar(problem):
    frontier = [(problem.heuristic(problem.initial_state), 0, problem.initial_state, [])]
    explored = set()
    nodes_expanded = 0

    while frontier:
        _, cost, state, path = heapq.heappop(frontier)
        if problem.goal_test(state):
            return path, nodes_expanded, len(frontier)

        if state not in explored:
            explored.add(state)
            nodes_expanded += 1

            for action in problem.actions(state):
                new_state = problem.result(state, action)
                if new_state not in explored:
                    heuristic_value = problem.heuristic(new_state)
                    heapq.heappush(frontier, (cost + 1 + heuristic_value, cost + 1, new_state, path + [action]))

    return None, nodes_expanded, len(frontier)

def main():
    for N in [4, 5, 6, 7, 8]:  # Test for all required values of N
        print(f"\nRunning for N = {N}")
        problem = ModifiedNQueensProblem(N)

        # BFS
        print("\nRunning BFS...")
        bfs_solution, bfs_nodes_expanded, bfs_frontier_size = bfs(problem)
        print(f"BFS Solution: {bfs_solution}")
        print(f"Nodes Expanded: {bfs_nodes_expanded}")
        print(f"Frontier Size: {bfs_frontier_size}")

        # UCS
        print("\nRunning UCS...")
        ucs_solution, ucs_nodes_expanded, ucs_frontier_size = ucs(problem)
        print(f"UCS Solution: {ucs_solution}")
        print(f"Nodes Expanded: {ucs_nodes_expanded}")
        print(f"Frontier Size: {ucs_frontier_size}")

        # A*
        print("\nRunning A*...")
        astar_solution, astar_nodes_expanded, astar_frontier_size = astar(problem)
        print(f"A* Solution: {astar_solution}")
        print(f"Nodes Expanded: {astar_nodes_expanded}")
        print(f"Frontier Size: {astar_frontier_size}")

if __name__ == '__main__':
    main()

#A* is the fastest because it uses a heuristic to guide the search.
#BFS and UCS do not use heuristics, so they take longer.



Running for N = 4

Running BFS...
BFS Solution: [(0, 'D'), (0, 'D'), (0, 'DR'), (1, 'DL'), (3, 'D'), (3, 'D')]
Nodes Expanded: 1448
Frontier Size: 6029

Running UCS...
UCS Solution: [(0, 'D'), (1, 'DR'), (1, 'DR'), (3, 'D'), (3, 'DL'), (3, 'DL')]
Nodes Expanded: 1764
Frontier Size: 6331

Running A*...
A* Solution: [(0, 'D'), (1, 'D'), (1, 'DR'), (1, 'DL'), (3, 'DL'), (3, 'DR')]
Nodes Expanded: 52
Frontier Size: 341

Running for N = 5

Running BFS...
BFS Solution: [(0, 'D'), (0, 'D'), (0, 'D'), (0, 'D'), (1, 'D'), (1, 'D'), (1, 'DR'), (2, 'DL'), (4, 'D'), (4, 'D')]
Nodes Expanded: 107127
Frontier Size: 497499

Running UCS...
UCS Solution: [(1, 'DR'), (2, 'DR'), (2, 'DR'), (3, 'D'), (3, 'DL'), (3, 'DL'), (4, 'D'), (4, 'DL'), (4, 'D'), (4, 'D')]
Nodes Expanded: 109074
Frontier Size: 498725

Running A*...
A* Solution: [(4, 'D'), (3, 'D'), (3, 'DL'), (3, 'DR'), (3, 'D'), (1, 'DR'), (1, 'DL'), (1, 'D'), (2, 'D'), (4, 'D')]
Nodes Expanded: 45
Frontier Size: 456

Running for N = 6

Running BF